# Лабораторная работа №3

## NUMBA

In [115]:
!pip install numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [116]:
!pip install tabulate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [117]:
from typing import Tuple, Mapping

import numpy as np
from matplotlib import pyplot as plt
import sklearn
import numba
from numba import jit
from datetime import datetime
from tabulate import tabulate

#### Тестовые функции

##### Функция трехгорбого верблюда
$$ {\displaystyle f(x,y)=2x^{2}-1.05x^{4}+{\frac {x^{6}}{6}}+xy+y^{2}} $$
Глобальный минимум
$$ {\displaystyle f(0,0)=0} $$
Метод поиска
$$ {\displaystyle -5\leq x,y\leq 5} $$

In [118]:
def test_func_Camel(x: np.ndarray, y:np.ndarray) -> np.float64:   
    return 2 * x**2 - 1.05 * x**4 + 1/6 * x**6 + x * y + y**2

def test_func_Camel_dx(x: np.ndarray, y:np.ndarray) -> np.float64:
    return 4 * x - 4.2 * x**3 + x**5 + y

def test_func_Camel_dy(x: np.ndarray, y:np.ndarray) -> np.float64:
    return x + 2 * y

##### Функция Изома
$$ {\displaystyle f(x,y)=-\cos \left(x\right)\cos \left(y\right)\exp \left(-\left(\left(x-\pi \right)^{2}+\left(y-\pi \right)^{2}\right)\right)} $$
Глобальный минимум
$$ {\displaystyle f(\pi ,\pi )=-1} $$
Метод поиска
$$ {\displaystyle -100\leq x,y\leq 100} $$

In [119]:
def test_func_Izom(x: np.ndarray, y:np.ndarray) -> np.float64:   
    return -np.cos(x) * np.cos(y) * np.exp(-((x - np.pi)**2 + (y - np.pi)**2))

def test_func_Izom_dx(x: np.ndarray, y:np.ndarray) -> np.float64:  
    return np.exp(-x**2-y**2-2*(np.pi)**2)*(np.cos(y)*np.exp(2*np.pi*x+2*np.pi*y)*np.sin(x)+(2*np.exp(2*np.pi*y)*np.cos(y)*x-2*np.pi*np.exp(2*np.pi*y)*np.cos(y))*np.exp(2*np.pi*x)*np.cos(x))

def test_func_Izom_dy(x: np.ndarray, y:np.ndarray) -> np.float64:
    return np.exp(-y**2-x**2-2*(np.pi)**2)*(np.cos(x)*np.exp(2*np.pi*y+2*np.pi*x)*np.sin(y)+(2*np.exp(2*np.pi*x)*np.cos(x)*y-2*np.pi*np.exp(2*np.pi*x)*np.cos(x))*np.exp(2*np.pi*y)*np.cos(y))

##### Функция Била
$$ {\displaystyle f(x,y)=\left(1.5-x+xy\right)^{2}+\left(2.25-x+xy^{2}\right)^{2}} $$
Глобальный минимум:
$$ {\displaystyle f(3,0.5)=0} $$
Метод поиска:
$$ {\displaystyle -4.5\leq x,y\leq 4.5} $$

In [120]:
def test_func_Bil(x: np.ndarray, y:np.ndarray) -> np.longdouble:
    return (1.5 - x + x*y)**2 + (2.25 - x + x*y**2)**2

def test_func_Bil_dx(x: np.ndarray, y:np.ndarray) -> np.longdouble:
    return (4 * y**4 - 4 * y**2 - 8*y + 8)*x * 0.5 + 4.5 *y**2 + 3 * y - 7.5

def test_func_Bil_dy(x: np.ndarray, y:np.ndarray) -> np.longdouble:
    return 4 * x**2 * y**3 + (9*x - 2*x**2)*y - 2 * x**2 + 3 * x

In [121]:
def my_gd(f: Mapping, dfdx: Mapping, dfdy: Mapping, x0: np.ndarray, lr: float = 0.001, T: int = 10000)-> Tuple [np.ndarray, np.ndarray, np.float32]:
    """
    Реализация градиентного спуска
    
    Args:
        f(Mapping) -> функционал для оптимизации
        df(Mapping) -> градиент оптимизирующего функционала (dx,dy)
        x0(np.ndarray) -> стартовая точка
        lr(float) -> скорость обучения
        T(int) -> количество итераций
    
    Returns:
        x, y, результат функции от этих x и y
    
    """
    
    x_old = x0[0]
    y_old = x0[1]
    
    for _ in range(T):
        x_new = x_old - lr * dfdx(x_old, y_old)
        y_new = y_old - lr * dfdy(x_old, y_old)
        
        x_old = x_new
        y_old = y_new
        #history.append([x_old.copy(), y_old.copy(), f(x_old, y_old)])
    return x_new, y_new, f(x_new, y_new)

## Реализация через NUMBA

In [122]:
@numba.njit(fastmath=True)
def test_func_Camel_numba(x: np.ndarray, y:np.ndarray) -> np.float64:   
    return 2 * x**2 - 1.05 * x**4 + 1/6 * x**6 + x * y + y**2

@numba.njit(fastmath=True)
def test_func_Camel_dx_numba(x: np.ndarray, y:np.ndarray) -> np.float64:
    return 4 * x - 4.2 * x**3 + x**5 + y

@numba.njit(fastmath=True)
def test_func_Camel_dy_numba(x: np.ndarray, y:np.ndarray) -> np.float64:
    return x + 2 * y

In [123]:
@numba.njit(fastmath=True)
def test_func_Izom_numba(x: np.ndarray, y:np.ndarray) -> np.float64:   
    return -np.cos(x) * np.cos(y) * np.exp(-((x - np.pi)**2 + (y - np.pi)**2))

@numba.njit(fastmath=True)
def test_func_Izom_dx_numba(x: np.ndarray, y:np.ndarray) -> np.float64:  
    return np.exp(-x**2-y**2-2*(np.pi)**2)*(np.cos(y)*np.exp(2*np.pi*x+2*np.pi*y)*np.sin(x)+(2*np.exp(2*np.pi*y)*np.cos(y)*x-2*np.pi*np.exp(2*np.pi*y)*np.cos(y))*np.exp(2*np.pi*x)*np.cos(x))

@numba.njit(fastmath=True)
def test_func_Izom_dy_numba(x: np.ndarray, y:np.ndarray) -> np.float64:
    return np.exp(-y**2-x**2-2*(np.pi)**2)*(np.cos(x)*np.exp(2*np.pi*y+2*np.pi*x)*np.sin(y)+(2*np.exp(2*np.pi*x)*np.cos(x)*y-2*np.pi*np.exp(2*np.pi*x)*np.cos(x))*np.exp(2*np.pi*y)*np.cos(y))

In [124]:
@numba.njit(fastmath=True)
def test_func_Bil_numba(x: np.ndarray, y:np.ndarray) -> np.longdouble:
    return (1.5 - x + x*y)**2 + (2.25 - x + x*y**2)**2

@numba.njit(fastmath=True)
def test_func_Bil_dx_numba(x: np.ndarray, y:np.ndarray) -> np.longdouble:
    return (4 * y**4 - 4 * y**2 - 8*y + 8)*x * 0.5 + 4.5 *y**2 + 3 * y - 7.5

@numba.njit(fastmath=True)
def test_func_Bil_dy_numba(x: np.ndarray, y:np.ndarray) -> np.longdouble:
    return 4 * x**2 * y**3 + (9*x - 2*x**2)*y - 2 * x**2 + 3 * x

In [125]:
@numba.njit(fastmath=True, debug=True)
def my_gd_numba(f: Mapping, dfdx: Mapping, dfdy: Mapping, x0: np.ndarray, lr: float = 0.001, T: int = 10000)-> Tuple [np.ndarray, np.ndarray, np.float32]:
    """
    Реализация градиентного спуска
    
    Args:
        f(Mapping) -> функционал для оптимизации
        df(Mapping) -> градиент оптимизирующего функционала (dx,dy)
        x0(np.ndarray) -> стартовая точка
        lr(float) -> скорость обучения
        T(int) -> количество итераций
    
    Returns:
        x, y, результат функции от этих x и y
    
    """
    
    x_old = x0[0]
    y_old = x0[1]
    
    for _ in range(T):
        x_new = x_old - lr * dfdx(x_old, y_old)
        y_new = y_old - lr * dfdy(x_old, y_old)
        
        x_old = x_new
        y_old = y_new
        #history.append([x_old.copy(), y_old.copy(), f(x_old, y_old)])
    return x_new, y_new, f(x_new, y_new)

## Сравнение времени

In [126]:
times = [[0,0], [0,0], [0,0]]

для функции трёхгорбого верблюда

In [127]:
start_time = datetime.now()
my_gd(test_func_Camel, test_func_Camel_dx, test_func_Camel_dy, np.array([-1,5]))
times[0][0] = datetime.now() - start_time

In [134]:
start_time = datetime.now()
my_gd_numba(test_func_Camel_numba, test_func_Camel_dx_numba, test_func_Camel_dy_numba, np.array([-1,5]))
times[0][1] = datetime.now() - start_time

для функции Изома

In [129]:
start_time = datetime.now()
my_gd(test_func_Izom, test_func_Izom_dx, test_func_Izom_dy, np.array([4,4]))
times[1][0] = datetime.now() - start_time

In [135]:
start_time = datetime.now()
my_gd_numba(test_func_Izom_numba, test_func_Izom_dx_numba, test_func_Izom_dy_numba, np.array([4,4]))
times[1][1] = datetime.now() - start_time

для функции Била

In [131]:
start_time = datetime.now()
my_gd(test_func_Bil, test_func_Bil_dx, test_func_Bil_dy, np.array([2, 0]))
times[2][0] = datetime.now() - start_time

In [136]:
start_time = datetime.now()
my_gd_numba(test_func_Bil_numba, test_func_Bil_dx_numba, test_func_Bil_dy_numba, np.array([2, 0]))
times[2][1] = datetime.now() - start_time

## Итоговая таблица с результатами замеров

In [137]:
data = [
    ["Без применения numba", times[0][0], times[1][0], times[2][0]],
    ["С применением numba",times[0][1], times[1][1], times[2][1]]
]

head = [" ", "Функция трёхгорбого верблюда", "Функция Изома", "Функция Била"]

print(tabulate(data, headers=head, tablefmt="grid"))

+----------------------+--------------------------------+-----------------+----------------+
|                      | Функция трёхгорбого верблюда   | Функция Изома   | Функция Била   |
+======================+================================+=================+================+
| Без применения numba | 0:00:00.036474                 | 0:00:00.359180  | 0:00:00.075418 |
+----------------------+--------------------------------+-----------------+----------------+
| С применением numba  | 0:00:00.000320                 | 0:00:00.005712  | 0:00:00.000495 |
+----------------------+--------------------------------+-----------------+----------------+
